In [97]:
import pyomo.environ as pyomo

# solver
solver = pyomo.SolverFactory('cbc',executable=r'C:\Cbc-master-win64-msvc16-mt\bin\cbc.exe')
solver.options['sec'] = 5

In [98]:
# read data 
import pandas as pd 
df = pd.read_csv('pyomo_data_larger_values_multiple_t.csv')
#df = df.iloc[6:12]
t_set = df['t'].unique()
i_set = df['i'].unique()

buyer 

In [99]:
# constraints 

# \sum_{i,t}Y_{i,t}<= Q^F_i 
def request(model):
    #return sum(model.Y_it[t] for t in model.T) <= (model.QF_i) + 1 
    return sum(model.Y_it[t] for t in model.T) <= (model.QF_i) 

#  f_{i,t}*g_{i,t} <= CO2_{i,t}
def carbon_target(model, t):  
    #if t==2: 
    #    return model.f_it[t] * model.g_it[t] <= model.CO2_it[t] + 1
    #else: 
        return model.f_it[t] * model.g_it[t] <= model.CO2_it[t] 
    
# Y_{i,t} + g_{i,t} + Q^S_{i,t} = D_{i,t} 
def demand(model, t):   
    #if t ==6: 
    #    return  (model.Y_it[t] + model.g_it[t] + model.QS_it[t] == model.D_it[t]+1) 
    #else: 
        return (model.Y_it[t] + model.g_it[t] + model.QS_it[t] == model.D_it[t]) 

#  g_{i,t} <= \sum^t_{t'=1} h_{i,t'}
def generator_capacity (model, t):  
    #if t == 6: 
    #      return model.g_it[t] <= sum(model.h_it[tp] for tp in model.T if tp <= t)+ 1
    #else: 
        return model.g_it[t] <= sum(model.h_it[tp] for tp in model.T if tp <= t)



In [ ]:
def buyer_model (): 
    lambda_dict = dict()
    for i in i_set :
        if i ==8: 
            # create a model 
            model = pyomo.ConcreteModel()

            # series 
            model.nt = pyomo.Param(initialize=len(df))
            model.T = pyomo.Set(initialize=t_set)

            # Parameters
            model.C_geninv_it = pyomo.Param(model.T, initialize={t: df.loc[(df['i'] == i) & (df['t'] == t), 'C_geninv_it'].values[0] for t in t_set})   # C^{geninv}_{i,t}
            model.C_gen_it = pyomo.Param(model.T, initialize={t: df.loc[(df['i'] == i) & (df['t'] == t), 'C_gen_it'].values[0] for t in t_set})         # C^{gen}_{i,t}
            model.f_it = pyomo.Param(model.T, initialize={t: df.loc[(df['i'] == i) & (df['t'] == t), 'f_it'].values[0] for t in t_set})                 # f_{i,t}
            model.CO2_it = pyomo.Param(model.T, initialize={t: df.loc[(df['i'] == i) & (df['t'] == t), 'CO2_it'].values[0] for t in t_set})             # CO2_{i,t}
            model.D_it = pyomo.Param(model.T, initialize={t: df.loc[(df['i'] == i) & (df['t'] == t), 'D_it'].values[0] for t in t_set})                 # D_{i,t}
            model.PS_it = pyomo.Param(model.T, initialize={t: df.loc[(df['i'] == i) & (df['t'] == t), 'PS_it'].values[0] for t in t_set})               # P^S_{i,t}
            model.PF_i = pyomo.Param(initialize=5.0)                                                                                                    # P^F_i = 1 for smaller values # P^F_i = 5 for larger values 

            # Variables
            model.g_it = pyomo.Var(model.T, domain=pyomo.NonNegativeReals)       # g_{i,t}
            model.h_it = pyomo.Var(model.T, domain=pyomo.NonNegativeReals)       # h_{i,t}
            model.QS_it = pyomo.Var(model.T, domain=pyomo.NonNegativeReals)      # Q^S_{i,t}
            model.Y_it = pyomo.Var(model.T, domain=pyomo.NonNegativeReals)       #Y_{i,t}
            model.QF_i = pyomo.Var(domain=pyomo.NonNegativeReals)                # Q^F_i

            # objective function 
            #min P^F_i * Q^F_i + \sum_t ( C^{geninv}_{i,t} * h_{i,t} + C^{gen}_{i,t} * g_{i,t} + P^S_{i,t} * Q^S_{i,t} )
            def objective_func(model):
                return model.PF_i * model.QF_i + \
                    sum(model.C_geninv_it[t] * model.h_it[t] + model.C_gen_it[t] * model.g_it[t] \
                        + model.PS_it[t] * model.QS_it[t] for t in model.T) 
            model.objective = pyomo.Objective(rule = objective_func, sense = pyomo.minimize)
            # constraint
            model.requestConstraint = pyomo.Constraint(rule=request)
            model.carbontargetConstraint = pyomo.Constraint(model.T, rule=carbon_target)
            model.demandConstraint = pyomo.Constraint(model.T, rule=demand) 
            model.demandConstraint = pyomo.Constraint(model.T, rule=demand) 
            model.generatorConstraint = pyomo.Constraint(model.T, rule=generator_capacity)

            # solve
            # dual 
            model.dual = pyomo.Suffix(direction=pyomo.Suffix.IMPORT)
            #objective     
            result = solver.solve(model)
            print(model.objective())
            print(result.solver.status)
            print(result.solver.termination_condition)    
            print(model.display())

            print("check KKT conditions")
            all_ok = True 

            #initiate list of lambda
            # no need dummy value because lambda1 is not dependent on t so no matching of dummy value is required        
            lambda1_list = []
            lambda_QF_list = []
            #dummy value so that the index of lambda matches t 
            lambda2_list = [""]
            lambda3_list = [""]
            lambda4_list = [""]        
            lambda_h_list = [""]
            lambda_g_list =[""]
            lambda_QS_list = [""]
            lambda_Y_list = [""]
                    
            for constr in model.component_objects(pyomo.Constraint, active=True):
                ok = True
                for idx in constr:
                    c = constr[idx]
                    val = pyomo.value(c.body)
                    lb, ub = c.lower, c.upper

                    # Get dual variable value (None if solver didn't return it)
                    λ = model.dual.get(c, None)   
                    #print(λ)         

                    # primal 
                    #print("primal")
                    if lb is not None and val < lb :
                        print(f"[Primal Infeasibility] {c.name}: {val} < {lb}")
                        all_ok = False
                    elif ub is not None and val > ub :
                        print(f"[Primal Infeasibility] {c.name}: {val} > {ub}")
                        all_ok = False
                    
                    # ---- dual ----
                    #print('dual')
                    if λ is not None:
                        if lb is None and ub is not None:  # ≤ constraint # neg 
                            if λ > 0:
                                print(f"[Dual Infeasibility] {c.name}: λ = {λ} > 0")
                                all_ok = False
                        elif lb is not None and ub is None:  # ≥ constraint #non-neg 
                            if λ < 0:
                                print(f"[Dual Infeasibility] {c.name}: λ = {λ} < 0")
                                all_ok = False
                        elif lb == ub:  # equality constraint — no sign restriction
                            if lb == 0 and λ != 0: # when D = 0 
                                print (f"[Dual Infeasibility] {c.name}: λ = {λ}" ) 
                            elif lb > 0 and λ <= 0 : # when D > 0 
                                print(f"[Dual Infeasibility] {c.name}: λ = {λ} - no sign restriction")
                            pass

                    if ok: 
                        if idx == None: # no time element #λ_1
                            lambda1_list.append (λ)
                        else: 
                            if str(constr) == "carbontargetConstraint": #λ_2
                                lambda2_list.append (λ)
                            elif str(constr) == "demandConstraint": #λ_3
                                lambda3_list.append (λ)
                            elif str(constr) == "generatorConstraint": #λ_4
                                lambda4_list.append (λ)
                            

            if all_ok == False:
                print("Some KKT conditions failed")
            print("done with KKT checks\n")
            print("λ_1", lambda1_list)
            print("λ_2", lambda2_list)
            print("λ_3", lambda3_list)
            print("λ_4", lambda4_list)

            lambda_dict ["λ_1"] = lambda1_list
            lambda_dict ["λ_2"] = lambda2_list
            lambda_dict ["λ_3"] = lambda3_list
            lambda_dict ["λ_4"] = lambda4_list

            f = 0 
            
            for t in range (1,(len(lambda_dict["λ_2"]))): 
                f += lambda_dict["λ_2"] [t] * model.CO2_it[t] + lambda_dict["λ_3"] [t] * model.D_it[t]   

                #use stationarity constraint to derive λ_h, λ_g, λ_QS, λ_Y      
                λ_h = float(model.C_geninv_it[t] - abs(lambda_dict ["λ_4"][t]))
                λ_g = float(model.C_gen_it[t] + abs(lambda_dict ["λ_2"][t])*model.f_it[t] - abs(lambda_dict ["λ_3"][t]) + abs(lambda_dict ["λ_4"][t]))
                λ_QS = float(model.PS_it[t] - abs(lambda_dict ["λ_3"][t]))
                λ_Y = float(abs(lambda_dict ["λ_1"][0]) - abs(lambda_dict ["λ_3"][t]))

                #checks for lambda derived from stationarity constraints
                #print(f"λ_h[{t}] = {float(model.C_geninv_it[t])} - {float(lambda_dict ["λ_4"][t])}")
                #print(f"λ_g [{t}] = {float(model.C_gen_it[t])} + {float(lambda_dict ["λ_2"][t]*model.f_it[t])} - {float(lambda_dict ["λ_3"][t])} + {float(lambda_dict ["λ_4"][t])}")
                #print(f"λ_QS[{t}]  = {float(model.PS_it[t])} - {float(lambda_dict ["λ_3"][t])}")
                #print(f"λ_Y [{t}] = {float(lambda_dict ["λ_1"][0])} - {float(lambda_dict ["λ_3"][t])}")

                if (λ_g < 0) or (float(pyomo.value(model.g_it[t])) < 0 ): #constraint 30 
                    all_ok = False
                if (λ_h < 0) or (float(pyomo.value(model.h_it [t])) < 0): #constraint 31 
                    all_ok = False            
                if (λ_QS < 0) or (float(pyomo.value(model.QS_it [t])) < 0):  #constraint 33 
                    all_ok = False 
                if (λ_Y < 0) or (float(pyomo.value(model.Y_it [t])) < 0): #constraint 34 
                    all_ok = False 

                lambda_g_list.append (λ_g)
                lambda_h_list.append (λ_h)
                lambda_QS_list.append (λ_QS) 
                lambda_Y_list.append(λ_Y) 

            print("Dual Objective Function", f)
            
            λ_QF = float(pyomo.value(model.PF_i) - lambda_dict ["λ_1"][0])
            lambda_QF_list.append(λ_QF)

            #constraint 32 
            if (λ_QF < 0) or (float(pyomo.value(model.QF_i)) <0): 
                all_ok = False

            print("λ_g_it",lambda_g_list)
            print("λ_h_it", lambda_h_list)
            print("λ_QF_i", lambda_QF_list)
            print("λ_QS_it", lambda_QS_list)
            print("λ_Y_it", lambda_Y_list)

            lambda_dict["λ_QF_i"]= lambda_QF_list
            lambda_dict["λ_h_it"] = lambda_h_list
            lambda_dict["λ_g_it"] =lambda_g_list
            lambda_dict["λ_QS_it"] = lambda_QS_list
            lambda_dict["λ_Y_it"] = lambda_Y_list
            print(lambda_dict)            

In [101]:
buyer_model()

(type=<class 'pyomo.core.base.constraint.IndexedConstraint'>) on block unknown
with a new Component (type=<class
'pyomo.core.base.constraint.IndexedConstraint'>). This is usually indicative
of a modelling error. To avoid this warning, use block.del_component() and
block.add_component().
122.0
ok
optimal
Model unknown

  Variables:
    g_it : Size=6, Index=T
        Key : Lower : Value : Upper : Fixed : Stale : Domain
          1 :     0 :   5.0 :  None : False : False : NonNegativeReals
          2 :     0 :   1.0 :  None : False : False : NonNegativeReals
          3 :     0 :   5.0 :  None : False : False : NonNegativeReals
          4 :     0 :   0.0 :  None : False : False : NonNegativeReals
          5 :     0 :   0.0 :  None : False : False : NonNegativeReals
          6 :     0 :   5.0 :  None : False : False : NonNegativeReals
    h_it : Size=6, Index=T
        Key : Lower : Value : Upper : Fixed : Stale : Domain
          1 :     0 :   5.0 :  None : False : False : NonNegative